# Run a full mission of basestation files


In [ ]:
import sys
import importlib
sys.path.append('/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1')
import warnings
warnings.simplefilter("ignore", category=Warning)

In [ ]:
from seagliderOG1 import readers
from seagliderOG1 import plotters
from seagliderOG1 import convertOG1
from seagliderOG1 import vocabularies
import os

In [ ]:
# To build the registry, use the following code:
if 0:
    mylist = fetchers.list_files_in_https_server(server)
    fetchers.create_pooch_registry_from_directory("/Users/eddifying/Dropbox/data/sg015-ncei-download/")
    # And then add the text to fetchers.py

In [ ]:
input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/"
#input_loc = "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20040924/"

def process_and_save_data(input_location, save=False, output_dir='../data'):
    # Load and concatenate all datasets from the server
    list_datasets = readers.read_basestation(input_location)
    
    # Convert the list of datasets to OG1
    ds_all = convertOG1.convert_to_OG1(list_datasets)
    
    if save:
        # Generate the output file path
        # Need to get the serial number of the platform for this
        output_file = os.path.join(output_dir, 'sg015_' + ds_all.start_date + '_delayed_temp.nc')
        
        # Check if the file exists and delete it if it does
        if os.path.exists(output_file):
            user_input = input(f"File {output_file} already exists. Do you want to delete it? (yes/no): ")
            if user_input.lower() != 'yes':
                print("File not deleted. Exiting the process.")
                return ds_all
            os.remove(output_file)
        
        # Save the dataset to a NetCDF file
        convertOG1.save_dataset(ds_all, output_file)
    
    return ds_all

# Example usage
ds_all = process_and_save_data(input_loc, save=True)